<a href="https://colab.research.google.com/github/castillosebastian/NLU_legal_domain/blob/master/NLU_proc_001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Script 1: procesamiento NLU

##  Entorno

In [3]:
# Libraries, tools and thanks 
import bs4
import nltk
import numpy
import pandas as pd
import io
import requests
from google.colab import drive
import numpy as np 
import json
import re

In [ ]:
# procesar con GPU

In [4]:
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Mounted at /content/drive


In [5]:
%cd  'drive/My Drive/Colab Notebooks/data'

/content/drive/My Drive/Colab Notebooks/data


In [6]:
%ls

corpus_fallosmetdat.json  tbdoctrina.json  tbmetdat.json
corpus_textosfallos.json  tbfallos.json


Lectura de tablas de datos primarios

In [7]:
with open('tbfallos.json', 'r') as myfile:
    data1=myfile.read()
# parse file
tbfallos = json.loads(data1)
tbfallos = pd.DataFrame.from_dict(tbfallos)

Instalar librería STANZA (Stanford NLP)

In [ ]:
! pip install stanza

In [ ]:
import stanza
stanza.download('es') # download spanish model

# Prueba de Tokenizacion de un sumario

In [37]:
fallos = tbfallos.loc[1:100, ['textos_fallo']] # selecting row and cols

In [38]:
fallos['textos_fallo'] = fallos.textos_fallo.str.replace("#","\n\n") # or fallos.textos_fallo.replace("#","\n\n", regex=True) 

In [39]:
corpus = fallos.textos_fallo.to_list()

In [40]:
corpus[1]

'\n\nTEXTO COMPLETO\n\n"VOSS SUSANA BEATRIZ C/ VILLARROEL MIRIAM ISABEL S/ REVISION DE COSA JUZGADA"\n\nLa Plata, 14 de Agosto de 2019.\n\nAUTOS Y VISTOS:\n\nLos señores Jueces doctores Torres, Soria y Genoud y la señora Jueza doctora Kogan dijeron:\n\nI. La Sala II de la Cámara Primera de Apelación en lo Civil y Comercial del Departamento Judicial de San Isidro confirmó la decisión que, a su tiempo, dispusiera la intervención del defensor oficial, en atención a la falta de presentación de los herederos de Susana Beatriz Voss (v. providencia simple electrónica de fecha 5-X-2018 y fs. 260 y vta.).\n\nII. Frente a ello, el defensor oficial interpone recurso extraordinario de inaplicabilidad de ley por el que aduce infracción a los arts. 33 inc. 2 de la ley 14.452; 375 inc. "d" del Código Civil y Comercial; 43, 53 inc. 5, 59, 60 y 341 del Código Procesal Civil y Comercial y 14, 16, 17, 18, 19, 28 y concs. de la Constitución nacional. Asimismo, aduce vulneración de doctrina legal que cita 

In [41]:
len(corpus)

100

In [ ]:
nlp = stanza.Pipeline(lang='es', processors='tokenize,mwt,pos,lemma', tokenize_no_ssplit=True)

Pruebas para procesamiento en lotes

In [67]:
doc = nlp(corpus[0])

In [72]:
doc0 = word_info_df(doc) # prueba sumario 1

In [73]:
len(doc0)

2013

In [74]:
doc0['fallo_nro'] = 0

In [75]:
doc0

,text,lemma,upos,xpos,fallo_nro
0,TEXTO,texto,NOUN,NOUN,0
1,COMPLETO,completo,ADJ,ADJ,0
2,A,a,ADP,ADP,0
3,C,c,NOUN,NOUN,0
4,U,U,CCONJ,CCONJ,0
...,...,...,...,...,...
2008,KOGAN,KOGAN,PROPN,PROPN,0
2009,CARLOS,CARLOS,PROPN,PROPN,0
2010,E.,E.,PROPN,PROPN,0
2011,CAMPS,CAMPS,PROPN,PROPN,0


In [76]:
doc = nlp(corpus[1]) 

In [77]:
doc1 = word_info_df(doc) # prueba sumario 0

In [78]:
len(doc1)

732

In [79]:
doc1['fallo_nro'] = 1

In [83]:
doc_union = pd.DataFrame() 

In [84]:
doc_union = pd.concat([doc0, doc1], axis=0)

In [85]:
doc_union

,text,lemma,upos,xpos,fallo_nro
0,TEXTO,texto,NOUN,NOUN,0
1,COMPLETO,completo,ADJ,ADJ,0
2,A,a,ADP,ADP,0
3,C,c,NOUN,NOUN,0
4,U,U,CCONJ,CCONJ,0
...,...,...,...,...,...
727,Secretario,Secretario,NOUN,NOUN,1
728,Registrada,registrado,ADJ,ADJ,1
729,bajo,bajo,ADP,ADP,1
730,el,el,DET,DET,1


In [43]:
def print_word_info(word):
    print(f"Text:\t{word.text}")
    print(f"Lemma: \t{word.lemma}")
    print(f"UPOS: \t{word.upos}")
    print(f"XPOS: \t{word.xpos}")

def word_info_df(doc):
    """
    - Parameters: doc (a Stanza Document object)
    - Returns: A Pandas DataFrame object with one row for each token in
      doc, and columns for text, lemma, upos, and xpos.
    """
    rows = []
    for sentence in doc.sentences:
        for word in sentence.words:
            row = {
                "text": word.text,
                "lemma": word.lemma,
                "upos": word.upos,
                "xpos": word.xpos,
            }
            rows.append(row)
    return pd.DataFrame(rows)



In [ ]:
word_info_df(doc)

In [92]:
df = pd.DataFrame()
counter = 0

for text in corpus:
  counter = counter + 1 
  temp = nlp(text)
  temp = word_info_df(temp)
  temp["fallo_nro"] = counter
  df = pd.concat([df, temp], axis=0)  

In [93]:
df

,text,lemma,upos,xpos,fallo_nro
0,TEXTO,texto,NOUN,NOUN,1
1,COMPLETO,completo,ADJ,ADJ,1
2,A,a,ADP,ADP,1
3,C,c,NOUN,NOUN,1
4,U,U,CCONJ,CCONJ,1
...,...,...,...,...,...
6905,SORIA,SORIA,PROPN,PROPN,100
6906,JUAN,JUAN,PROPN,PROPN,100
6907,JOSE,JOSE,PROPN,PROPN,100
6908,MARTIARENA,MARTIARENA,PROPN,PROPN,100


In [ ]:
def print_word_info(word):
    print(f"Text:\t{word.text}")
    print(f"Lemma: \t{word.lemma}")
    print(f"UPOS: \t{word.upos}")
    print(f"XPOS: \t{word.xpos}")

print_word_info(moby_p1.sentences[3].words[4])

# Text:     growing
# Lemma:    grow
# UPOS:     VERB
# XPOS:     VBG

def word_info_df(doc):
    """
    - Parameters: doc (a Stanza Document object)
    - Returns: A Pandas DataFrame object with one row for each token in
      doc, and columns for text, lemma, upos, and xpos.
    """
    rows = []
    for sentence in doc.sentences:
        for word in sentence.words:
            row = {
                "text": word.text,
                "lemma": word.lemma,
                "upos": word.upos,
                "xpos": word.xpos,
            }
            rows.append(row)
    return pd.DataFrame(rows)

word_info_df(moby_p1)

# 	text	lemma	upos	xpos
# 0	Call    call	VERB	VB
# 1	me      I   	PRON	PRP
# 2	Ishmael Ishmael	PROPN	NNP
# 3	.       .       PUNCT   .
# 4	Some	some	DET     DT

In [ ]:
nlp = stanza.Pipeline(lang='es', processors='tokenize,mwt,pos,lemma', tokenize_no_ssplit=True)

In [21]:
corpus = nlp(corpus)

In [22]:
def print_doc_info(doc):
    print(f"Num sentences:\t{len(doc.sentences)}")
    print(f"Num tokens:\t{doc.num_tokens}")
    print(f"Num words:\t{doc.num_words}")
    print(f"Num entities:\t{len(doc.entities)}")

In [36]:
print_doc_info(corpus_list)

Num sentences:	110
Num tokens:	6875
Num words:	6910
Num entities:	0
